In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np
import qiskit
%matplotlib inline
import random

from qiskit import IBMQ
api_token = ''
IBMQ.enable_account(api_token)
backend_list = IBMQ.backends()
print(backend_list)
backend_sim = IBMQ.get_backend('ibmq_qasm_simulator')


Alice_01=[]
Alice_basis=[]
Bob_basis=[]
Bob_01=[]
Bob_key=[]
all_right_times=0
all_total_times=0
Eve_basis=[]
Bob_key_after_Eve=[]
all_right_times_after_Eve=0
all_total_times_after_Eve=0

[<IBMQBackend('ibmqx4') from IBMQ()>, <IBMQBackend('ibmqx2') from IBMQ()>, <IBMQBackend('ibmq_16_melbourne') from IBMQ()>, <IBMQSimulator('ibmq_qasm_simulator') from IBMQ()>]


In [20]:
def random_01str(list,n):
    for i in range(n):
        list.append(random.randint(0,1))

In [21]:
def runBB84(nn):
    n=nn
    
    #Alice
    alice_1=[]
    random_01str(alice_1,n)
    #print("0 or 1 qubit : {}".format(alice_1))

    alice_2=[]
    random_01str(alice_2,n)
    #print("Z or X basis : {}".format(alice_2))

    # Create a Quantum Register with n qubits.
    q = QuantumRegister(n,'q')

    # Create a Quantum Circuit acting on the q register
    A_circ = QuantumCircuit(q)

    for i in range(n):
        if alice_1[i]==1:
            A_circ.x(q[i])
        
    for j in range(n):
        if alice_2[j]==1:
            A_circ.h(q[j])

    A_circ.barrier(q)
    #A_circ.draw()
    
    #Bob
    bob_1=[]

    bob_2=[]
    random_01str(bob_2,n)
    #print("measured by Z(0) or X(1) : {}".format(bob_2))

    c = ClassicalRegister(n,'c')
    B_circ = QuantumCircuit(q,c)

    for i in range(n):
        if bob_2[i]==1:
            B_circ.h(q[i])

    B_circ.measure(q,c)
    qc = A_circ + B_circ
    #qc.draw()
    
    job_sim = qiskit.execute(qc, backend_sim, shots=5)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(qc)
    #print(counts)
    #from qiskit.visualization import plot_histogram
    #plot_histogram(counts)
    
    result_str = list(counts)[0]
    #print("IBMQ_result = {}".format(result_str))

    bob_1.clear()
    for i in range(n):
        bob_1.append(int(result_str[n-1-i]))
    #print("Bob_result = {}".format(bob_1))
    
    key1=[]
    pos=[]
    for i in range(n):
        if alice_2[i]==bob_2[i]:
            key1.append(bob_1[i])
            pos.append(i)
    #print("the same basis : {}".format(pos))
    #print("key1 = {}".format(key1))
    key2=key1
    
    #Check
    #print("key1 = {}".format(key2))
    #print("the same basis : {}".format(pos))
    num=len(key2)
    total_times=len(key2)//2
    right_times = 0

    for i in range(total_times):
        rand=random.randint(0,num-1)
        if alice_1[pos[rand]]==bob_1[pos[rand]]:
            right_times+=1
            #print("check pos = {}, ".format(pos[rand]),end=' ')
            #print("Alice {} = Bob {}".format(alice_1[pos[rand]],bob_1[pos[rand]]))
        else :
            #print("check pos = {}".format(pos[rand]),end=' ')
            #print("Alice {} != Bob {}".format(alice_1[pos[rand]],bob_1[pos[rand]]))
            print("different qubit has been found")
        #print("rand = {}".format(rand),end=' ')
        #print("check pos = {}".format(pos[rand]))
        del key2[rand]
        del pos[rand]
        num-=1
    
    #print("key2 = {}".format(key2))
    for i in range(len(key2)):       ###new-------------------------------------------------
        global Bob_key
        Bob_key.append(key2[i])
    acc=(right_times/total_times)
    print("accuracy = {}%".format(acc*100))
    global all_right_times           ###new-------------------------------------------------
    all_right_times+=right_times
    global all_total_times
    all_total_times+=total_times
    
    #Eve
    eve_2=[]
    random_01str(eve_2,n)
    #print("measured by Z(0) or X(1) : {}".format(eve_2))
    
    q2 = QuantumRegister(n,'q2')
    E_circ = QuantumCircuit(q2)

    for i in range(n):
        if alice_2[i]==eve_2[i]:
            if alice_1[i]==1:
                E_circ.x(q2[i])
            if alice_2[i]==1:
                E_circ.h(q2[i])
        else :
            if alice_2[i]==0 and eve_2[i]==1:
                rand2 = random.randint(0,1)
                if not rand2:
                    E_circ.h(q2[i])
                else :
                    E_circ.x(q2[i])
                    E_circ.h(q2[i])
            elif alice_2[i]==1 and eve_2[i]==0:
                rand2 = random.randint(0,1)
                if rand2:
                    E_circ.x(q2[i])
             
    E_circ.barrier(q2)
    #E_circ.draw()
    
    #After Eve, sended to Bob
    c2 = ClassicalRegister(n,'c2')
    B_circ2 = QuantumCircuit(q2,c2)

    for i in range(n):
        if bob_2[i]==1:
            B_circ2.h(q2[i])

    B_circ2.measure(q2,c2)
    qc2 = E_circ + B_circ2
    #qc2.draw()
    
    job_sim2 = qiskit.execute(qc2, backend_sim, shots=5)
    result_sim2 = job_sim2.result()
    counts2 = result_sim2.get_counts(qc2)
    #print(counts2)
    #from qiskit.visualization import plot_histogram
    #plot_histogram(counts2)
    
    result_str2 = list(counts2)[0]
    #print("IBMQ_result2 = {}".format(result_str2))

    bob_1.clear()
    for i in range(n):
        bob_1.append(int(result_str2[n-1-i]))  
    #print("Bob_result2 = {}".format(bob_1))
    
    key1_2=[]
    pos2=[]
    for i in range(n):
        if alice_2[i]==bob_2[i]:
            key1_2.append(bob_1[i])
            pos2.append(i)
    #print("the same basis : {}".format(pos2))
    #print("key1_2 = {}".format(key1_2))
    key2_2=key1_2
    
    #Check
    #print("key1_2 = {}".format(key2_2))
    #print("the same basis : {}".format(pos2))
    num2=len(key2_2)
    total_times2=len(key2_2)//2
    right_times2 = 0

    for i in range(total_times2):
        rand=random.randint(0,num2-1)
        if alice_1[pos2[rand]]==bob_1[pos2[rand]]:
            right_times2+=1
            #print("check pos = {}, ".format(pos2[rand]),end=' ')
            #print("Alice {} = Bob {}".format(alice_1[pos2[rand]],bob_1[pos2[rand]]))
        else :
            #print("check pos = {}".format(pos2[rand]),end=' ')
            #print("Alice {} != Bob {}".format(alice_1[pos2[rand]],bob_1[pos2[rand]]))
            print("different qubit has been found")
        #print("rand = {}".format(rand),end=' ')
        #print("check pos2 = {}".format(pos2[rand]))
        del key2_2[rand]
        del pos2[rand]
        num2-=1
    
    #print("key2_2 = {}".format(key2_2))
    for i in range(len(key2_2)):             ###new-------------------------------------------------
        global Bob_key_after_Eve
        Bob_key_after_Eve.append(key2_2[i])
    acc2=(right_times2/total_times2)
    print("accuracy_2 = {}%".format(acc2*100))  
    global all_right_times_after_Eve          ###new-------------------------------------------------
    all_right_times_after_Eve+=right_times2
    global all_total_times_after_Eve
    all_total_times_after_Eve+=total_times2
    
    #Check all
    key1_2=[]
    pos2=[]
    for i in range(n):
        if alice_2[i]==bob_2[i]:
            key1_2.append(bob_1[i])
            pos2.append(i)
    key2_2=key1_2

    num2=len(key2_2)
    total_times2=len(key2_2)
    right_times2 = 0

    for i in range(total_times2):
        rand=random.randint(0,num2-1)
        if alice_1[pos2[rand]]==bob_1[pos2[rand]]:
            right_times2+=1
            #print("check pos = {} ".format(pos2[rand]),end=', ')
            #print("Alice {} = Bob {}".format(alice_1[pos2[rand]],bob_1[pos2[rand]]))
        else :
            #print("check pos = {} ".format(pos2[rand]),end=', ')
            #print("Alice {} != Bob {}".format(alice_1[pos2[rand]],bob_1[pos2[rand]]))
            print("different qubit has been found")
        #print("rand = {}".format(rand),end=' ')
        #print("check pos2 = {}".format(pos2[rand]))
        del key2_2[rand]
        del pos2[rand]
        num2-=1
    
    #print("key2_2 = {}".format(key2_2))
    acc2=(right_times2/total_times2)
    print("accuracy_2 = {}%".format(acc2*100))   




In [22]:
n=70 #number of qubit which Alice creates
numq=n//16
numr=n%16
#print("numq=",numq)
#print("numr=",numr)
times=numq
if numr!=0:
    times+=1
#print("times=",times)
    
def runFor(n,times):
    n2=n
    #nn=0
    for i in range(times):
        if n2>=16:
            n2-=16
            nn=16
        else :
            nn=n2
        #print("n=",n2)
        #print("nn=",nn)
        runBB84(nn)
        
        
runFor(n,times)

print("Bob_key = {}".format(Bob_key))

accuracy=(all_right_times/all_total_times)
print("accuracy = {}%".format(accuracy*100))

print("Bob_key_after_Eve = {}".format(Bob_key_after_Eve))

accuracy_after_Eve=(all_right_times_after_Eve/all_total_times_after_Eve)
print("accuracy_after_Eve = {}%".format(accuracy_after_Eve*100))

accuracy = 100.0%
different qubit has been found
different qubit has been found
accuracy_2 = 60.0%
different qubit has been found
different qubit has been found
different qubit has been found
accuracy_2 = 70.0%
accuracy = 100.0%
different qubit has been found
accuracy_2 = 75.0%
different qubit has been found
different qubit has been found
different qubit has been found
accuracy_2 = 62.5%
accuracy = 100.0%
different qubit has been found
accuracy_2 = 80.0%
different qubit has been found
different qubit has been found
different qubit has been found
accuracy_2 = 72.72727272727273%
accuracy = 100.0%
accuracy_2 = 100.0%
accuracy_2 = 100.0%
accuracy = 100.0%
accuracy_2 = 100.0%
accuracy_2 = 100.0%
Bob_key = [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1]
accuracy = 100.0%
Bob_key_after_Eve = [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
accuracy_after_Eve = 80.0%
